In [1]:
import pandas as pd
import numpy as np
import sys, time
sys.path.append("../") # go to parent dir
from src.data.diversity_document import DiversityDocument

from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.word2vec import Word2Vec
import gensim.utils
import itertools
import pyLDAvis.gensim
from IPython.display import clear_output

df = pd.read_csv('../data/external/2017-18stats.csv')
with open('../data/external/download_whitelist.txt', 'r') as f:
    whitelist = [line.strip() for line in f]
df['CompanyLinkToGPGInfo'] = df['CompanyLinkToGPGInfo'].astype(str)

df.describe()


,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,MaleLowerQuartile,FemaleLowerQuartile,MaleLowerMiddleQuartile,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile
count,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000
mean,14.470046,11.963943,15.259415,-13.792430,35.844000,34.328861,46.427434,53.572566,50.636047,49.363953,54.995196,45.004804,60.959228,39.040772
std,15.087346,15.687637,242.144758,1258.967565,36.398861,36.102908,24.114317,24.114317,26.063711,26.063711,26.172304,26.172304,24.316170,24.316170
min,-400.000000,-164.000000,-21092.000000,-120695.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.300000,0.900000,0.000000,0.000000,1.000000,0.400000,27.000000,35.550000,29.600000,27.500000,34.000000,21.200000,41.800000,17.050000
50%,13.500000,9.600000,22.500000,5.700000,20.500000,18.000000,44.400000,55.600000,48.500000,51.500000,54.000000,46.000000,63.000000,37.000000
75%,23.000000,21.100000,53.000000,38.300000,73.800000,70.900000,64.450000,73.000000,72.500000,70.400000,78.800000,66.000000,82.950000,58.200000
max,159.000000,75.700000,3848.200000,5566.700000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [2]:
#split the corpus into quartiles by the "best" companies on diversity numbers. 
upper_q = df.DiffMeanHourlyPercent.quantile(0.80)
lower_q = df.DiffMeanHourlyPercent.quantile(0.20)
df_bad = df.loc[df.DiffMeanHourlyPercent > upper_q]
df_good = df.loc[df.DiffMeanHourlyPercent < lower_q]
print('Number of companies in worst quarter: %s' % df_bad.DiffMeanHourlyPercent.count())
print('Number of companies in best quarter: %s' % df_good.DiffMeanHourlyPercent.count())

Number of companies in worst quarter: 2026
Number of companies in best quarter: 2029


In [3]:
def iter_diversity_data(df_divers):
    number_rows = df_divers.shape[0]
    counter = 0
    for index, row in df_divers.iterrows():
        counter = counter + 1
        percent_complete = (counter / number_rows) * 100
    
        if counter % 100 == 0:
            #clear_output(wait=True)
            print('%.2f percent complete' % (percent_complete))        

        doc = DiversityDocument(row)

        if doc.url_hash in whitelist:
            continue

        if not doc.has_downloaded_file and not doc.company_link == 'nan':
            doc.download()
            if doc.raw_text is None:
                with open('../data/external/download_whitelist.txt', 'a') as whitelist_file:
                    whitelist_file.write(doc.url_hash + '\n')    

        if doc.raw_text is not None and doc.clean_text is not None:
            yield doc.clean_text

    print('Processing Finished: 100%')        
    

In [4]:
class CorpusDiversity(object):
    def __init__(self, df):
        self.df = df
        self.dictionary = Dictionary(iter_diversity_data(df))

    def __iter__(self):
        for doc in iter_diversity_data(self.df):
            # tokenize each message; simply lowercase & match alphabetic chars, for now
            yield self.dictionary.doc2bow(doc)
           
#    def __len__(self):
#        return len(self.dictionary)
   
class CorpusSentanceDiversity(object):
    def __init__(self, df):
        self.df = df
        self.dictionary = Dictionary(iter_diversity_data(df))

    def __iter__(self):
        for doc in iter_diversity_data(self.df):
            # tokenize each message; simply lowercase & match alphabetic chars, for now
            yield doc    

bad_tokenized_corpus = CorpusDiversity(df_bad)
good_tokenized_corpus = CorpusDiversity(df_good)

bad_sentance_corpus = CorpusSentanceDiversity(df_bad)
good_sentance_corpus = CorpusSentanceDiversity(df_good)

4.94 percent complete
9.87 percent complete
14.81 percent complete
19.74 percent complete
24.68 percent complete
29.62 percent complete
34.55 percent complete
39.49 percent complete
44.42 percent complete
49.36 percent complete
54.29 percent complete
59.23 percent complete
64.17 percent complete
69.10 percent complete
74.04 percent complete
78.97 percent complete
83.91 percent complete
88.85 percent complete
93.78 percent complete
98.72 percent complete
Processing Finished: 100%
4.93 percent complete
9.86 percent complete
14.79 percent complete
19.71 percent complete
24.64 percent complete
29.57 percent complete
34.50 percent complete
39.43 percent complete
44.36 percent complete
49.29 percent complete
54.21 percent complete
59.14 percent complete
64.07 percent complete
69.00 percent complete
73.93 percent complete
78.86 percent complete
83.79 percent complete
88.71 percent complete
93.64 percent complete
98.57 percent complete
Processing Finished: 100%
4.94 percent complete
9.87 perce

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [5]:
bad_corpus = bad_tokenized_corpus
good_corpus = good_tokenized_corpus

# Add bigrams and trigrams to docs (only ones that appear 5 times or more).
#bigram = Phrases(docs, min_count=5)
#for idx in range(len(docs)):
#    for token in bigram[docs[idx]]:
#        if '_' in token:
            # Token is a bigram, add to document.
#            docs[idx].append(token)

# Create a dictionary representation of the documents, and filter out frequent and rare words.
bad_dictionary = bad_tokenized_corpus.dictionary
good_dictionary = good_tokenized_corpus.dictionary

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
bad_dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
bad_dictionary.compactify()
good_dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
good_dictionary.compactify()

# This sort of "initializes" dictionary.id2token.
_ = bad_dictionary[0]
_ = good_dictionary[0]


In [6]:
print('Number of unique good tokens: %d' % len(good_dictionary))
print('Number of unique bad tokens: %d' % len(bad_dictionary))

Number of unique good tokens: 7371
Number of unique bad tokens: 9175


In [7]:
bad_lda_model = LdaModel(corpus=bad_corpus,
               id2word=bad_dictionary,
               num_topics=12, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

good_lda_model = LdaModel(corpus=good_corpus,
               id2word=good_dictionary,
               num_topics=12, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

4.94 percent complete
9.87 percent complete
14.81 percent complete
19.74 percent complete
24.68 percent complete
29.62 percent complete
34.55 percent complete
39.49 percent complete
44.42 percent complete
49.36 percent complete
54.29 percent complete
59.23 percent complete
64.17 percent complete
69.10 percent complete
74.04 percent complete
78.97 percent complete
83.91 percent complete
88.85 percent complete
93.78 percent complete
98.72 percent complete
Processing Finished: 100%
4.94 percent complete
9.87 percent complete
14.81 percent complete
19.74 percent complete
24.68 percent complete
29.62 percent complete
34.55 percent complete
39.49 percent complete
44.42 percent complete
49.36 percent complete
54.29 percent complete
59.23 percent complete
64.17 percent complete
69.10 percent complete
74.04 percent complete
78.97 percent complete
83.91 percent complete
88.85 percent complete
93.78 percent complete
98.72 percent complete
Processing Finished: 100%
4.94 percent complete
9.87 perce

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [8]:
# Compute Perplexity
bad_lda_corpus = [x for x in bad_corpus]
good_lda_corpus = [x for x in good_corpus]
# a measure of how good the model is. lower the better. -7.670575002821739
print('\nGood Perplexity: ', bad_lda_model.log_perplexity(bad_lda_corpus)) 
print('\nBad Perplexity: ', good_lda_model.log_perplexity(good_lda_corpus)) 

4.94 percent complete
9.87 percent complete
14.81 percent complete
19.74 percent complete
24.68 percent complete
29.62 percent complete
34.55 percent complete
39.49 percent complete
44.42 percent complete
49.36 percent complete
54.29 percent complete
59.23 percent complete
64.17 percent complete
69.10 percent complete
74.04 percent complete
78.97 percent complete
83.91 percent complete
88.85 percent complete
93.78 percent complete
98.72 percent complete
Processing Finished: 100%
4.93 percent complete
9.86 percent complete
14.79 percent complete
19.71 percent complete
24.64 percent complete
29.57 percent complete
34.50 percent complete
39.43 percent complete
44.36 percent complete
49.29 percent complete
54.21 percent complete
59.14 percent complete
64.07 percent complete
69.00 percent complete
73.93 percent complete
78.86 percent complete
83.79 percent complete
88.71 percent complete
93.64 percent complete
98.57 percent complete
Processing Finished: 100%

Good Perplexity:  -7.2742743954

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [18]:
good_documents = list(good_sentance_corpus) 
bad_documents = list(bad_sentance_corpus) 

good_w2vmodel = Word2Vec (good_documents, size=150, window=10, min_count=2, workers=10)
good_w2vmodel.train(good_documents,total_examples=len(good_documents),epochs=10)

bad_w2vmodel = Word2Vec (bad_documents, size=150, window=10, min_count=2, workers=10)
bad_w2vmodel.train(bad_documents,total_examples=len(bad_documents),epochs=10)

4.93 percent complete
9.86 percent complete
14.79 percent complete
19.71 percent complete
24.64 percent complete
29.57 percent complete
34.50 percent complete
39.43 percent complete
44.36 percent complete
49.29 percent complete
54.21 percent complete
59.14 percent complete
64.07 percent complete
69.00 percent complete
73.93 percent complete
78.86 percent complete
83.79 percent complete
88.71 percent complete
93.64 percent complete
98.57 percent complete
Processing Finished: 100%
4.94 percent complete
9.87 percent complete
14.81 percent complete
19.74 percent complete
24.68 percent complete
29.62 percent complete
34.55 percent complete
39.49 percent complete
44.42 percent complete
49.36 percent complete
54.29 percent complete
59.23 percent complete
64.17 percent complete
69.10 percent complete
74.04 percent complete
78.97 percent complete
83.91 percent complete
88.85 percent complete
93.78 percent complete
98.72 percent complete
Processing Finished: 100%


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


(8064278, 10007960)

In [21]:
com = 'diversity'
print(good_w2vmodel.wv.most_similar(positive=com))
print(bad_w2vmodel.wv.most_similar(positive=com))

[('fund', 0.6929117441177368), ('local', 0.6766580939292908), ('partnership', 0.6601231098175049), ('futures', 0.653863787651062), ('communities', 0.646407961845398), ('wellbeing', 0.6353886723518372), ('customised', 0.6330523490905762), ('reinvest', 0.6305705904960632), ('patient', 0.627884566783905), ('county', 0.6169164776802063)]
[('charitable', 0.6746270656585693), ('partnership', 0.6643285155296326), ('serve', 0.6572934985160828), ('local', 0.6504461169242859), ('stakeholder', 0.6310160160064697), ('engagement', 0.6070657968521118), ('active', 0.6024819016456604), ('charity', 0.5930672883987427), ('volunteer', 0.5929437875747681), ('philanthropy', 0.5652163624763489)]


In [11]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(bad_lda_model, bad_lda_corpus, bad_dictionary)

/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      22.633827        1       1 -0.175752 -0.216854
8      14.384036        1       2 -0.187611 -0.106251
4      14.112637        1       3 -0.075167  0.163735
0      12.810280        1       4 -0.184888 -0.068289
10      9.579188        1       5  0.395755 -0.173231
9       6.338068        1       6 -0.024884  0.111501
5       6.204747        1       7 -0.085258  0.137338
2       5.286824        1       8 -0.019528  0.142184
1       3.204338        1       9  0.180835  0.076646
3       2.767125        1      10  0.069015  0.211367
6       2.025985        1      11 -0.051101 -0.205615
11      0.652949        1      12  0.158585 -0.072531, topic_info=     Category         Freq                                   Term  \
term                                                                
459   Default  5743.000000                                  bonus   
173   Default  5534.000000                                 female   
189   Default  4243.000000                                   male   
536   Default  4071.000000                               quartile   
515   Default  3305.000000                                 median   
190   Default  3919.000000                                    man   
192   Default  3659.000000                                   mean   
277   Default  2850.000000                              diversity   
327   Default  1956.000000                               services   
793   Default  2374.000000                                    new   
69    Default  1158.000000                                    ltd   
1024  Default  1340.000000                                  trust   
67    Default  1404.000000                                limited   
1448  Default  1231.000000                                 school   
1538  Default  1426.000000                                     U�   
71    Default  2718.000000                             management   
214   Default  3427.000000                                   role   
534   Default  1927.000000                             proportion   
512   Default  2325.000000                                    low   
166   Default  1698.000000                               equality   
498   Default  1540.000000                              inclusion   
180   Default  1772.000000                                 hourly   
217   Default  2641.000000                                service   
2422  Default  1109.000000                                     K�   
642   Default  1398.000000                                  staff   
517   Default  1932.000000                                 middle   
78    Default  1774.000000                                   news   
549   Default  2264.000000                                 senior   
4714  Default  1039.000000                                   cmyk   
337   Default  1597.000000                                    use   
...       ...          ...                                    ...   
9163  Topic12    47.122330                     Hertfordshire EN11   
9165  Topic12    47.122330  Inclusion and Respect in Construction   
9168  Topic12    47.122330                                 WISE     
9170  Topic12    47.122330                            compensator   
9169  Topic12    41.900402                              adjusters   
7686  Topic12    38.433987                                   tile   
9161  Topic12    36.678474        Corporate Responsibility Health   
927   Topic12    32.947395                                   drug   
9173  Topic12    31.456543               Accessibility     Modern   
6810  Topic12    27.545307                                  27.9%   
9166  Topic12    26.234617                 Projects Back Projects   
9167  Topic12    26.234617                      Projects Projects   
9158  Topic12    26.234617                                    Fax   
9174  Topic12    26.234613 

In [12]:
pyLDAvis.gensim.prepare(good_lda_model, good_lda_corpus, good_dictionary)

/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      20.832119        1       1 -0.151323 -0.060063
6      19.693521        1       2 -0.278297  0.122840
0      13.388009        1       3  0.076532 -0.166212
1       8.775666        1       4  0.124725 -0.099380
5       7.189196        1       5  0.005942 -0.150106
9       5.451174        1       6  0.256975  0.386610
7       4.845516        1       7  0.131055 -0.176547
10      4.699025        1       8 -0.014709 -0.004192
2       4.622651        1       9 -0.059323  0.016465
8       4.523990        1      10 -0.079437  0.007324
4       3.996789        1      11 -0.223820  0.114126
11      1.982345        1      12  0.211681  0.009134, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
794   Default  2972.000000         care  2972.000000  30.0000  30.0000
943   Default  3458.000000       female  3458.000000  29.0000  29.0000
925   Default  3514.000000        bonus  3514.000000  28.0000  28.0000
965   Default  2989.000000     quartile  2989.000000  27.0000  27.0000
951   Default  2998.000000         male  2998.000000  26.0000  26.0000
980   Default  3391.000000        woman  3391.000000  25.0000  25.0000
954   Default  2455.000000       median  2455.000000  24.0000  24.0000
552   Default  2623.000000         mean  2623.000000  23.0000  23.0000
1780  Default  1007.000000      council  1007.000000  22.0000  22.0000
952   Default  2398.000000          man  2398.000000  21.0000  21.0000
94    Default  1614.000000          job  1614.000000  20.0000  20.0000
4093  Default   787.000000       arriva   787.000000  19.0000  19.0000
548   Default  1863.000000          low  1863.000000  18.0000  18.0000
947   Default  1578.000000       hourly  1578.000000  17.0000  17.0000
104   Default  1208.000000      limited  1208.000000  16.0000  16.0000
176   Default  1877.000000        staff  1877.000000  15.0000  15.0000
73    Default  1441.000000         find  1441.000000  14.0000  14.0000
979   Default  1189.000000        upper  1189.000000  13.0000  13.0000
359   Default  1660.000000         rate  1660.000000  12.0000  12.0000
185   Default  2724.000000           uk  2724.000000  11.0000  11.0000
784   Default  1361.000000      average  1361.000000  10.0000  10.0000
119   Default  1406.000000         news  1406.000000   9.0000   9.0000
955   Default  1255.000000       middle  1255.000000   8.0000   8.0000
1093  Default   661.000000    colleague   661.000000   7.0000   7.0000
53    Default  1385.000000    diversity  1385.000000   6.0000   6.0000
172   Default  1475.000000     services  1475.000000   5.0000   5.0000
1153  Default  1071.000000   proportion  1071.000000   4.0000   4.0000
275   Default   805.000000        event   805.000000   3.0000   3.0000
118   Default  1347.000000          new  1347.000000   2.0000   2.0000
371   Default  1644.000000         role  1644.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
7243  Topic12    28.416767        downs    29.447392   3.8853  -5.7805
6601  Topic12    27.849941     sterling    28.880489   3.8846  -5.8007
7248  Topic12    27.836897         inch    28.867476   3.8845  -5.8011
3421  Topic12    27.473713         sets    28.504240   3.8841  -5.8143
6618  Topic12    27.384909   motorcycle    28.415438   3.8839  -5.8175
2899  Topic12   242.977020  accessories   252.588531   3.8821  -3.6345
4810  Topic12   164.276260         cars   173.469543   3.8664  -4.0259
1989  Topic12    87.382240       repair    97.478607   3.8115  -4.6572
3971  Topic12   178.262955          car   218.539993   3.7172  -3.9442
7249  Topic12    44.372131       island    47.984741   3.8426  -5.3349
3415  Topic12    80.703873       offers    94.835579   3.7595  -4.7367
4830  Topic12   194.274338       garden   269.017853   3.5954  -3.8582
992   